## Fine-tuning tutorial for Evo2
This tutorial goes through a toy fine-tuning example end to end starting with a fasta and continuing training a hugging
face checkpoint on this user defined dataset.

In [9]:
# Clean up any prior runs
!rm -rf preprocessed_data
!rm -rf preatraining_demo
!rm -rf nemo2_evo2_1b_8k
!rm -rf pretraining_demo
!rm -rf training_data_config.yaml
!rm -rf preprocess_config.yaml
!rm -f chr17.fa.gz
!rm -f chr18.fa.gz
!rm -f chr21.fa.gz
!rm -f chr17.fa
!rm -f chr18.fa
!rm -f chr21.fa
!rm -f chr17_18_21.fa


In [2]:
import os
concat_path = "chr17_18_21.fa"
if not os.path.exists(concat_path):
    !wget https://hgdownload.soe.ucsc.edu/goldenpath/hg38/chromosomes/chr17.fa.gz
    !wget https://hgdownload.soe.ucsc.edu/goldenpath/hg38/chromosomes/chr18.fa.gz
    !wget https://hgdownload.soe.ucsc.edu/goldenpath/hg38/chromosomes/chr21.fa.gz
    !zcat chr17.fa.gz > chr17.fa
    !zcat chr18.fa.gz > chr18.fa
    !zcat chr21.fa.gz > chr21.fa
    !cat chr17.fa chr18.fa chr21.fa > chr17_18_21.fa


--2025-02-25 01:11:46--  https://hgdownload.soe.ucsc.edu/goldenpath/hg38/chromosomes/chr17.fa.gz
Resolving hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)|128.114.119.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25930986 (25M) [application/x-gzip]
Saving to: ‘chr17.fa.gz.2’

chr17.fa.gz.2       100%[===================>]  24.73M  82.3MB/s    in 0.3s    

2025-02-25 01:11:49 (82.3 MB/s) - ‘chr17.fa.gz.2’ saved [25930986/25930986]

--2025-02-25 01:11:49--  https://hgdownload.soe.ucsc.edu/goldenpath/hg38/chromosomes/chr18.fa.gz
Resolving hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)|128.114.119.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25154367 (24M) [application/x-gzip]
Saving to: ‘chr18.fa.gz.1’

chr18.fa.gz.1       100%[===================>]  23.99M

In [3]:
full_fasta_path = os.path.abspath(concat_path)
output_dir = os.path.abspath("preprocessed_data")
output_yaml = f"""
- datapaths: ["{full_fasta_path}"]
  output_dir: "{output_dir}"
  output_prefix: chr17_18_21_uint8_distinct
  train_split: 0.9
  valid_split: 0.05
  test_split: 0.05
  overwrite: True
  embed_reverse_complement: true
  random_reverse_complement: 0.0
  random_lineage_dropout: 0.0
  include_sequence_id: false
  transcribe: "back_transcribe"
  force_uppercase: false
  indexed_dataset_dtype: "uint8"
  tokenizer_type: "Byte-Level"
  vocab_file: null
  vocab_size: null
  merges_file: null
  pretrained_tokenizer_model: null
  special_tokens: null
  fast_hf_tokenizer: true
  append_eod: true
  enforce_sample_length: null
  ftfy: false
  workers: 1
  preproc_concurrency: 100000
  chunksize: 25
  drop_empty_sequences: true
  nnn_filter: false  # If you split your fasta on NNN (in human these are contigs), then you should set this to true.
  seed: 12342  # Not relevant because we are not using random reverse complement or lineage dropout.
"""
with open("preprocess_config.yaml", "w") as f:
    print(output_yaml, file=f)


In [4]:
!preprocess_evo2 --config preprocess_config.yaml

[WARNING  | bitsandbytes.cextension]: Could not find the bitsandbytes CUDA binary at PosixPath('/usr/local/lib/python3.12/dist-packages/bitsandbytes/libbitsandbytes_cuda128.so')
[WARNING  | bitsandbytes.cextension]: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
[WARNING  | py.warnings        ]: /usr/local/lib/python3.12/dist-packages/mamba_ssm/ops/selective_scan_interface.py:163: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd

[WARNING  | py.warnings        ]: /usr/local/lib/python3.12/dist-packages/mamba_ssm/ops/selective_scan_interface.py:239: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd

[WARNING  | py.warnings        ]: /usr/local/lib/python3.12/dist-packages/mamba_ssm/

In [5]:
!ls -lh preprocessed_data/

total 402M
-rw-r--r-- 1 ubuntu ubuntu 159M Feb 25 01:12 chr17_18_21_uint8_distinct_byte-level_test.bin
-rw-r--r-- 1 ubuntu ubuntu   82 Feb 25 01:12 chr17_18_21_uint8_distinct_byte-level_test.idx
-rw-r--r-- 1 ubuntu ubuntu 154M Feb 25 01:12 chr17_18_21_uint8_distinct_byte-level_train.bin
-rw-r--r-- 1 ubuntu ubuntu   82 Feb 25 01:12 chr17_18_21_uint8_distinct_byte-level_train.idx
-rw-r--r-- 1 ubuntu ubuntu  90M Feb 25 01:12 chr17_18_21_uint8_distinct_byte-level_val.bin
-rw-r--r-- 1 ubuntu ubuntu   82 Feb 25 01:12 chr17_18_21_uint8_distinct_byte-level_val.idx


In [6]:
!evo2_convert_to_nemo2 \
  --model-path hf://arcinstitute/savanna_evo2_1b_base \
  --model-size 1b --output-dir nemo2_evo2_1b_8k

[WARNING  | bitsandbytes.cextension]: Could not find the bitsandbytes CUDA binary at PosixPath('/usr/local/lib/python3.12/dist-packages/bitsandbytes/libbitsandbytes_cuda128.so')
[WARNING  | bitsandbytes.cextension]: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
[NeMo W 2025-02-25 01:12:44 nemo_logging:405] /usr/local/lib/python3.12/dist-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
      warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
    
[WARNING  | py.warnings        ]: /usr/local/lib/python3.12/dist-packages/mamba_ssm/ops/selective_scan_interface.py:163: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd

[WARNING  | py.warnings        ]: /usr/local/lib/python3.1

In [7]:
from pathlib import Path
output_pfx = str(Path(os.path.abspath("preprocessed_data"))/"chr17_18_21_uint8_distinct_byte-level")
output_yaml = f"""
- dataset_prefix: {output_pfx}_train
  dataset_split: train
  dataset_weight: 1.0
- dataset_prefix: {output_pfx}_val
  dataset_split: validation
  dataset_weight: 1.0
- dataset_prefix: {output_pfx}_test
  dataset_split: test
  dataset_weight: 1.0
"""
with open("training_data_config.yaml", "w") as f:
    print(output_yaml, file=f)

In [8]:
# For evo2 training and fine-tuning follow the same set of steps, so we use the same train_evo2 command.
#  the big difference is the --ckpt-dir argument which points to a pre-existing checkpoint from some other training run.
!train_evo2 \
    -d training_data_config.yaml \
    --dataset-dir {preprocessed_data} \
    --experiment-dir pretraining_demo \
    --model-size 1b \
    --devices 1 \
    --num-nodes 1 \
    --seq-length 1024 \
    --micro-batch-size 2 \
    --lr 0.0001 \
    --warmup-steps 5 \
    --max-steps 100 \
    --ckpt-dir nemo2_evo2_1b_8k \
    --clip-grad 1 \
    --wd 0.01 \
    --activation-checkpoint-recompute-num-layers 1 \
    --val-check-interval 50 \
    --ckpt-async-save \
    --no-wandb

[WARNING  | bitsandbytes.cextension]: Could not find the bitsandbytes CUDA binary at PosixPath('/usr/local/lib/python3.12/dist-packages/bitsandbytes/libbitsandbytes_cuda128.so')
[WARNING  | bitsandbytes.cextension]: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
[NeMo W 2025-02-25 01:13:17 nemo_logging:405] /usr/local/lib/python3.12/dist-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
      warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
    
[WARNING  | py.warnings        ]: /usr/local/lib/python3.12/dist-packages/mamba_ssm/ops/selective_scan_interface.py:163: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd

[WARNING  | py.warnings        ]: /usr/local/lib/python3.1